In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
from io import BytesIO
from PIL import Image, ImageDraw
import os
import ftplib
import pandas as pd
from glob import glob
import imageio.v3 as iio


In [2]:
#defining start and end time
start = '2023-08-15'
end = '2023-08-28'

 

In [3]:
#loading data
d = xr.open_dataset('/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/ERA5_SouthAmerica_2023_hourly_tcwv.nc')

#subsetting data to average over i
d = d.sel(time = slice(start, end))
d = d['tcwv']

#defining number of frames in given time period
num = len(d['time'])
print(num)
print(d)

336
<xarray.DataArray 'tcwv' (time: 336, latitude: 181, longitude: 341)>
[20738256 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -140.0 -139.8 -139.5 ... -55.5 -55.25 -55.0
  * latitude   (latitude) float32 -15.0 -15.25 -15.5 ... -59.5 -59.75 -60.0
  * time       (time) datetime64[ns] 2023-08-15 ... 2023-08-28T23:00:00
Attributes:
    units:          kg m**-2
    long_name:      Total column vertically-integrated water vapour
    standard_name:  lwe_thickness_of_atmosphere_mass_content_of_water_vapor


In [4]:
#data vis and projection
cmap = cmocean.cm.diff_r
lev = np.arange(0, 30, 0.25)
proj = ccrs.PlateCarree(central_longitude=-77)

#creating loop
for i in range(0, num):

    fig = plt.figure(figsize = (9, 4.5), dpi = 300)
    ax = plt.axes(projection = proj)
    
    # filled contour map of TPWV
    d[i].plot.contourf(
        x = 'longitude',
        y = 'latitude',
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='both',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":"Total Precipitable Water Vapor (kg/m^2)"})
    # add coastlines
    ax.coastlines(
       resolution='50m')  #Currently can be one of “110m”, “50m”, and “10m”.

# add coastlines masking the land in lightgray
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='k', facecolor='lightgray'))

# add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='gray',
                  alpha=0.5,
                  linestyle='--')

#extract year, month, day, and hour from the time series using splits, time format YYY-MM-DD-T00:00:00.000000000
    times = d['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str)).split('-')[1]
    day = str(np.datetime64(times, 'D').astype(str)).split('-')[2]
    hr = str(np.datetime64(times, 'h').astype(str)).split('T')[1].split(':')[0]

# add title
    ax.set_title("TCWV on "+ day + mon + ', ' + yr + ' at ' + hr + ':00')
#constructing filename
    name = f'RiversTestTCWV_{yr}-{mon}-{day}-{hr}.png'

    #saveFolder = '/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/AtmoRiversFrames'
    
    
# save figure 
    fig.savefig('AtmoRiversTCWVFrames/' + name, facecolor = 'white', transparent = False, bbox_inches ='tight')
   # fig.savefig(name, facecolor = 'white', transparent = False, bbox_inches ='tight')

    plt.close(fig)
    #print(name)
     
    

In [5]:
#Making GIFs

#directory = '/AtmoriversIVTFrames/'
#filenames = 'RiversTestIVT*.png'


files = sorted(glob('AtmoRiversTCWVFrames/RiversTestTCWV*.png'))
files



['AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-00.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-01.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-02.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-03.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-04.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-05.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-06.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-07.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-08.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-09.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-10.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-11.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-12.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-13.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-14.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-15.png',
 'AtmoRiversTCWVFrames/RiversTestTCWV_2023-08-15-16.png',
 'AtmoRiversTC

In [6]:

#creating array
images = [ ] 

#setting loop 
for filename in files:
        images.append(iio.imread(filename))

iio.imwrite('RiversTestTCWV.gif', images, duration = 100, loop = 0)
